In [37]:
using Knet, Gym, Random

In [85]:
w = Knet.load("w_montezuma_vdiff.jld2","model")
summary(w)

"10-element Array{Array{Float32,N} where N,1}"

In [39]:
function preprocess(I)
    I = I[36:195,:,:]
    I = I[1:2:end, 1:2:end, 1]
    I[I .== 144] .= 0
    I[I .== 109] .= 0
    #I[I .!= 0] .= 1
    return vec(I)
end

preprocess (generic function with 1 method)

In [40]:
function predict(w, x)
    x = Float32.(x)
    x = reshape(x, 80, 80, 1, :)
    x = relu.(conv4(w[1], x, stride=4, padding=2) .+ w[2])
    x = relu.(conv4(w[3], x, stride=2) .+ w[4])
    x = mat(x)
    x = relu.(w[5]*x .+ w[6])
    prob_act = w[7] * x .+ w[8]
    value = w[9] * x .+ w[10]
    return prob_act, value
end

function sample_action(probs)
    probs = Array(probs)
    #probs[1] = 0
    
    cprobs = cumsum(probs, dims=1)
    sampled = cprobs .> rand() 
    actions = mapslices(argmax, sampled, dims=1)
    return actions[1]
end

sample_action (generic function with 1 method)

In [86]:
env = Gym.GymEnv("MontezumaRevenge-v0")
xsize = (80, 80, 1) # input dimensions of the policy network
amap = Dict(1=>0, 2=>1, 3=>2, 4=>3, 5=>4, 6=>5, 7=>11, 8=>12) #0:NOP, 1:JUMP, 2:UP, 3:RIGHT, 4:LEFT, 5:DOWN, 11:RIGHT JUMP, 12:LEFT JUMP     
seed = 27



seed > 0 && (Random.seed!(seed); Gym.seed!(env, seed))

for episode=1:15
    state = Gym.reset!(env)
    render(env)
    episode_reward = 0

    prev_s = zeros(prod(xsize))
    while true
        curr_s = preprocess(state)
        s = curr_s .- prev_s
        prev_s = curr_s

        p, V = predict(w, s)
        p = softmax(p, dims=1)
        action = sample_action(p)
        state, reward, done, info = Gym.step!(env, amap[action]) 
        
        render(env)
        sleep(0.05)    
        episode_reward += reward
        done && break
    end
      
    println("Episode:$episode, Reward:$episode_reward")
end
Gym.close!(env)
 
    

InterruptException: InterruptException: